In [4]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [5]:
train_x = np.load('ex5_train_x.npy')
train_y_original = np.load('ex5_train_y.npy')
train_y = train_y_original.astype('uint8')
#train_x
print(train_y.max())
print(train_x.shape, train_x.dtype)
print(train_y.shape, train_y.dtype)

5
(1020, 64, 64, 3) uint8
(1020,) uint8


In [6]:
#split data
def split(x,y,test_portion):
    #try shuffle here?
    size = x.shape[0]    
    train_size = int(size * (1-test_portion))
    x_train = x[0:train_size,:,:,:]
    y_train = y[0:train_size]
    print(x_train.shape, y_train.shape)

    test_size = size - train_size
    x_test = x[train_size:,:,:,:]
    y_test = y[train_size:]

    print(x_test.shape, y_test.shape)

    return train_size, x_train, y_train, test_size, x_test, y_test

In [7]:
def one_hot_encoding(y):
    length = y.shape[0]
    #print(length)
    output = np.zeros((6,length))      
    for i in range(length):         
        if(y[i][0]>5):  #to prevent index out of bound
            print('label is wrong');
            return -1
        else:
            #print(y[i])
            output[y[i][0]][i]=1   
    return output

In [8]:
train_size, x_train, y_train, test_size, x_test, y_test = split(train_x,train_y,0.2)

#one_col = y_train.reshape(y_train.shape[0],1)
#y_train_one_hot = one_hot_encoding(one_col)

X_train = x_train/255  # do i need mean center here??
X_test = x_test/255
Y_train = one_hot_encoding(y_train.reshape(y_train.shape[0],1)).T
Y_test = one_hot_encoding(y_test.reshape(y_test.shape[0],1)).T

print(Y_train.shape)
print(Y_test.shape)


(816, 64, 64, 3) (816,)
(204, 64, 64, 3) (204,)
(816, 6)
(204, 6)


In [9]:
def activation_forward(z, activation_index=1):      
    activation_set = ["sigmoid","ReLU"]    
    if(activation_index >= len(activation_set)):
        print('no such function')
        return -1       
    #sigmoid
    if activation_index == 0:  
        #check z shape?
        #print(z.shape)
        a =  1.0/(1.0 + np.exp(-z))
        #a = np.divide(1, np.add (1 , np.exp(-z)))        
    #ReLU
    else:
        a = np.maximum(z, np.zeros(z.shape)) #will take max(z[i],0) 
    return a


def activation_backward(da, a,z,function_index=1,alp=1):
    activation_set = ["sigmoid","ReLU","tanh","Leaky ReLU","ELU"]
    
    if(function_index >= len(activation_set)):
        print('no such function')
        return -1
    
    if function_index== 0:
        dadz = a * (1 - a)
    elif function_index == 1:
        dadz = np.ones(z.shape)        
        dadz[z < 0] = 0 
    elif function_index == 2:
        dadz = 1 - a*a    
    elif function_index==3:
        dadz = np.ones(z.shape)
        dadz[z < 0] = alp 
    else:
        dadz = np.ones(z.shape)
        dadz[z < 0] = alp * np.exp(dadz[z < 0]) 
    if(da.shape!=dadz.shape):
        print("shape not match in activation backward")
        dz = -1
    else:
        dz = np.multiply(da,dadz)
        
    return dz        

In [10]:
#test and plots for activation functions
# z1 = np.arange(-5,5,0.1)

# a1 = activation_forward(z1,0)
# plt.plot(z1, a1)
# plt.title('sigmoid')
# plt.show()

# a2 = activation_forward(z1,1)
# plt.plot(z1, a2)
# plt.title('ReLU')
# plt.show()

In [11]:
def calculate_cross_entropy_loss(a, i=0, j=1020):
    # Y_train is global variable
    if(a.shape==Y_train[i:j,:].shape):         
        if(a.any()==0):
            #print ('shift to right')
            a[a==0] = 0.00000001            
        if(a.any()==1):
            #print ('shift to left') 
            a[a==1] = 0.99999999                      
        temp1 = Y_train[i:j,:] * np.log(a)
        temp2 = (1 - Y_train[i:j,:]) * np.log(1 - a)  #could be wrong here in cost / loss ?
        #print ("total cost %f" %np.sum(temp1 + temp2))
        #print ("i j", i, j, "batch size", j-i)
        loss = (-1 / (j-i)) * np.sum(temp1 + temp2)  #j must > i         
    else:
        print('I did something wrong, shape of a is ', a.shape, ' it should be ', Y_train[i:j,:].shape)
        loss = -1
    return loss

def accuracy(prediction,label):
    if (prediction.shape!=label.shape):
        print ('wrong input size')
        print (prediction.shape)
        print (label.shape)        
        return -1
    count = 0
    total = prediction.shape[0]
    for i in range(total):        
        if prediction[i]!= label[i]:
            count+=1
    #print 'error count', count
    result = 0.0
    error_rate = float((count)/total)
    accuracy = 1-error_rate
    print ("accuracy", accuracy)
    return accuracy

In [12]:
import h5py
def save_model(para, path):
    f = h5py.File(path,'w')
    for k, v in para.items():
        f.create_dataset(k, data=v)
    f.close()
    print("file saved")

def load_model(path):
    dataset = h5py.File(path,'r')
    dataset.keys()
    para = {}
    for i in dataset.keys():
        para[i] = np.array(dataset[i])
        #print(para[i].shape)
    dataset.close()
    return para

def print_model(parameters):
    print("relu","max","relu","max","sigmoid","sigmoid")
    for k, v in parameters.items():
        print (k,v.shape)


In [13]:
def zero_pad (x,n):
    #x_padded = np.pad(x, (n,), 'constant') #could be wrong here
    x_padded = np.pad(x, ((0, 0), (n, n), (n, n), (0, 0)), 'constant')
    return x_padded

In [14]:
def single_convolution_step(a,kernel,b):
    if(a.shape!=kernel.shape):
        print ("shape do not match:")
        print(a.shape)
        print(kernel.shape)
        return -1
    else:
        #print ("shape match:")
        #print(a.shape, kernel.shape)
        #print ("conv sum", np.sum(np.multiply(a,kernel)) + b  )
        return np.sum(np.multiply(a,kernel)) + b   
 
## test
# a=np.arange(1,7).reshape(2,3)
# b=np.arange(3,9).reshape(2,3)
# print(a,"\n",b)
# print(single_convolution_step(a,b,2))

# d3 = np.ones((3,2,2))
# print(d3)
# np.sum(d3, axis=0)

def convolution_forward_one_layer(x,kernel,bias,stride=1,pad=2):
    batch_size = x.shape[0]  
    x_h = x.shape[1]
    x_w = x.shape[2]
    x_c = x.shape[3]   
    f_h = kernel.shape[0] #switch here??
    f_w = kernel.shape[1] #switch here??
    #print(f_w,f_h)
    channel = kernel.shape[3]    
    w = int((x_w-f_w+pad*2)/stride) + 1
    h = int((x_h-f_h+pad*2)/stride) + 1  
    
    x_pads = zero_pad(x, pad)
    #x_pads = zero_pad(A_prev, pad)
    output_4d = np.zeros((batch_size,h,w,channel))    
    #print(output_4d.shape)
    output_3d = np.zeros((batch_size,w,h))
    
    for batch in range(batch_size):
        x_pad = x_pads[batch]
#         print("x_pad shape", x_pad.shape)
#         print (kernel.shape)
        for j in range(h):
            for i in range(w):
                for c in range(channel):              
                    #print("conv forward: ", batch, j, i, c)
                    #print("h begin, h end, w begin, w end:",j*stride, j*stride+f_h, i*stride, i*stride+f_w)
                    output_4d[batch][j][i][c]= single_convolution_step(x_pad[j*stride:j*stride+f_h, i*stride:i*stride+f_w,:], kernel[...,c], bias[...,c])
        #output_3d[batch] = np.sum(output_4d[batch], axis=0)
         
    assert(output_4d.shape == (batch_size, h,w, channel))
    
    cache = (x, kernel, bias, channel, stride, pad)

    return output_4d, cache

    #return output_3d

In [15]:
def pooling_forward_one_layer (x,stride=1,f_w=2,f_h=2,function = "max"):

    batch_size, x_h, x_w, x_c = x.shape
    #print(batch_size, x_h, x_w, x_c)
    
    w = int((x_w-f_w)/stride) + 1
    h = int((x_h-f_h)/stride) + 1  
    channel = x_c
    #print(w,h,channel)
    
    output_4d = np.zeros((batch_size, h,w,channel))
           
    for batch in range(batch_size):
        for j in range(h):
            for i in range(w):
                for c in range(channel):              
                    x_slice = x[batch, j*stride:j*stride+f_h, i*stride:i*stride+f_w,c]
                    if(function =="max"):
                        output_4d[batch][j][i][c]= np.max(x_slice)
                    elif(function == "ave"):
                        output_4d[batch][j][i][c]= np.mean(x_slice)
      
    assert(output_4d.shape == (batch_size, h,w, channel))
    
    cache = (x, channel, stride, f_w, f_h )

    return output_4d, cache


In [16]:
def max_mask(x):
    
    return (x==np.max(x))

# np.random.seed(1)
# x = np.random.randn(3,2)
# mask = max_mask(x)
# print('x = ', x)
# print("mask = ", mask)

In [17]:
def distribute (dp, h, w):
     
    return np.ones((h,w)) * dp / float(h*w)

# a = distribute(2, 2, 2)
# print('distributed value =', a)


In [18]:
def pooling_backward_one_layer (da, cache, function = "max"):
    
    #cache = (x, channel, stride, f_w, f_h )
    x_prev, channel, stride, f_w, f_h = cache    
    batch_size, x_h_prev, x_w_prev, x_c_prev = x_prev.shape
    _,h,w,channel = da.shape    
    da_prev = np.zeros(x_prev.shape)
    
    for batch in range(batch_size):
        x_p = x_prev[batch]
        for j in range(h):
            for i in range(w):
                for c in range(channel):                       
                    
                    if(function =="max"):
                        #output_4d[batch][j][i][c]= np.max(x_slice)
                        x_p_slice = x_p[j*stride:j*stride+f_h, i*stride:i*stride+f_w,c]
                        mask = max_mask(x_p_slice)
                        da_prev[batch, j*stride:j*stride+f_h, i*stride:i*stride+f_w,c] += np.multiply(mask, da[batch][j][i][c])
                        
                    elif(function == "ave"):
                        #output_4d[batch][j][i][c]= np.mean(x_slice)                        
                        da_p = da[batch][j][i][c]
                        da_prev[batch, j*stride:j*stride+f_h, i*stride:i*stride+f_w,c] += distribute(da_p, f_h, f_w)
   
    assert(da_prev.shape == x_prev.shape)
    return da_prev

In [19]:
def convolution_backward_one_layer(dz,cache):
         
    x_prev, kernel, bias, channel, stride, pad = cache  
    (batch_size, x_h_prev, x_w_prev, x_c_prev) = x_prev.shape
    (f_h, f_w, channel_prev, channel) = kernel.shape  
    _, x_h, x_w, x_c = dz.shape
    
    output = np.zeros((batch_size,x_h_prev, x_w_prev, x_c_prev))                           
    dw = np.zeros((f_h, f_w, channel_prev, channel))
    db = np.zeros((1, 1, 1, channel))
    x_prev_pad = zero_pad(x_prev, pad)
    dx_prev_pad = zero_pad(output, pad)
    
    for batch in range(batch_size):      
        a_prev_pad = x_prev_pad[batch]
        da_prev_pad = dx_prev_pad[batch]    
        for j in range(x_h):
            for i in range(x_w):
                for c in range(x_c):  
                    a_slice = a_prev_pad[j*stride:j*stride+f_h, i*stride:i*stride+f_w,:]                     
                    da_prev_pad[j*stride:j*stride+f_h, i*stride:i*stride+f_w,:] += kernel[...,c] * dz[batch][j][i][c]
                   
                    dw[:,:,:,c] += a_slice * dz[batch][j][i][c]
                    db[:,:,:,c] += dz[batch][j][i][c]                    
       
        output[batch, :, :, :] = da_prev_pad[pad:-pad, pad:-pad, :]   
    
    assert(output.shape == (batch_size, x_h_prev, x_w_prev, x_c_prev))
    
    return output, dw, db

In [20]:
def initialization(function=1):
    
    np.random.seed(0)
    #s = np.sum(shape)
    c= 0.0  
    if (function== 0):
        c = math.sqrt(2)
    elif (function== 2):
        c = 4 * math.sqrt(2)
    else:
        c = 2    
    #print ('sigma,', c)    
    
    shape = np.zeros((6,4))
    shape[1] = (4, 4, 3, 8)
    shape[2] = (1, 1, 3, 8)
    shape[3] = (2, 2, 8, 16)
    shape[4] = (1, 1, 8, 16)
    #shape[5] = (16, 6)
    #shape[6] = (1, 6)    
        
    w1 = np.random.normal(0, c / math.sqrt(10000) ,(4, 4, 3, 8))
    w2 = np.random.normal(0, c / math.sqrt(10000) ,(4, 4, 8, 16))
    
    w3 = np.random.normal(0, c / math.sqrt(10000) ,(4096, 6))
#     w3 = np.random.normal(0, c / math.sqrt(10000) ,(4096, 256))
    w4 = np.random.normal(0, c / math.sqrt(10000) ,(256, 6))
    
    b1 = np.random.normal(0, c / math.sqrt(10000) ,(1, 1, 1, 8))
    b2 = np.random.normal(0, c / math.sqrt(10000) ,(1, 1, 1, 16))
    
    b3 = np.random.normal(0, c / math.sqrt(10000) ,(1, 6))
#     b3 = np.random.normal(0, c / math.sqrt(10000) ,(1, 256))
    b4 = np.random.normal(0, c / math.sqrt(10000) ,(1, 6))
       
    parameters = {"w1": w1,
                  "w2": w2,
                  "w3": w3,
                  "w4": w4,
                  "b1": b1,
                  "b2": b2,
                  "b3": b3,
                  "b4": b4
                 }
    
    return parameters    

In [21]:
temp_para = initialization(1)
temp_para.keys()

dict_keys(['w1', 'w2', 'w3', 'w4', 'b1', 'b2', 'b3', 'b4'])

In [22]:
def forward_propagation(X, parameters):
        
    w1 = parameters["w1"]
    w2 = parameters["w2"]
    w3 = parameters["w3"]
    w4 = parameters["w4"]
    
    b1 = parameters["b1"]
    b2 = parameters["b2"]
    b3 = parameters["b3"]
    b4 = parameters["b4"]
            
    
    #def convolution_forward_one_layer(x,kernel,bias,stride=1,pad=2):
    Z1, c_cache1 = convolution_forward_one_layer(X,w1,b1,2,2)
    A1 = activation_forward(Z1,1)
    P1, p_cache1 = pooling_forward_one_layer (A1,1,2,2,"max")   #change to parameters later

    Z2, c_cache2 = convolution_forward_one_layer(P1,w2,b2,2,2)
    A2 = activation_forward(Z2,1)
    P2, p_cache2 = pooling_forward_one_layer (A2,1,2,2,"max")
    P2_flat = P2.reshape(P2.shape[0],P2.shape[1]*P2.shape[2]*P2.shape[3])
    #print(P2.shape, P2_flat.shape)
    
    Z3 = np.dot(P2_flat,w3) + b3
    A3 = activation_forward(Z3,0)  
    
    Z4 = Z3
    A4 = A3
    
#     Z4 = np.dot(A3, w4) + b4
#     A4 = activation_forward(Z4,0) 
    
    # last layer need to be sigmoid !!!
    
    
    all_cache = { 
                  "z1": Z1,
                  "c_cache1": c_cache1,
                  "a1": A1,        
                  "p1": P1,
                  "p_cache1": p_cache1,
        
                  "z2": Z2,
                  "c_cache2": c_cache2,
                  "a2": A2,
                  "p2": P2,
                  "p_cache2": p_cache2,            
                  
                  "p2_flat": P2_flat,
                  "z3": Z3,
                  "a3": A3,
        
                  "z4": Z4,
                  "a4": A4
                 }
    
    # MAXPOOL: window 4x4, stride 4, padding 'SAME'
    #P2 = tf.nn.max_pool(A2, ksize = [1,4,4,1], strides = [1,4,4,1], padding = 'SAME')
    # FLATTEN
    
    # FULLY-CONNECTED without non-linear activation function (not not call softmax).
    # 6 neurons in output layer. Hint: one of the arguments should be "activation_fn=None" 
    
    #Z3 = tf.contrib.layers.fully_connected(P2, 6, activation_fn=None)

    return A4, all_cache

In [23]:
def lastLayerGradient(a, X, i=0, j=1020):  #i j can be used for mini batch, X is the input to last layer
    if(a.shape==Y_train[i:j,:].shape):     
        dldz = a - Y_train[i:j,:]
        m = j-i
        dldw = np.dot(np.transpose(X), dldz)/(m) # m or -m?
        dldb = np.sum(dldz, axis = 0, keepdims = True)/(m)
        #print("in lastLayerGradient",dldw.shape, dldb.shape)
        return dldz, dldw, dldb
    else:
        print('I did something wrong, shape of a is ', a.shape, ' it should be ', Y_train[i:j,:].shape)
        return -1

In [24]:
def backward_propagation(parameters, cache, begin,end, learning_rate = 0.01):
        
     
    #dldz_4, dldw_4, dldb_4 = lastLayerGradient(cache["a4"] , cache["a3"] )    
    dldz_3, dldw_3, dldb_3 = lastLayerGradient(cache["a3"] , cache["p2_flat"], begin, end)    #might be wrong here
    
    # need do backward for fully connected layer
    #times weight??? w3
    #print("dldz_3 shape",dldz_3.shape)
    da = np.dot(parameters["w3"], dldz_3.T) 
    #print("da shape", da.shape)
    
    
#     print("w1 shape", parameters["w1"].shape)
#     print("w2 shape", parameters["w2"].shape)
#     print("w3 shape", parameters["w3"].shape)
#     print("b1 shape", parameters["b1"].shape)
#     print("b2 shape", parameters["b2"].shape)
#     print("b3 shape", parameters["b3"].shape)

 
    
    #reshape here?
    p2_shape = cache["p2"].shape
    #print("p2_shape", p2_shape)
    dA3 = da.reshape(p2_shape[0],p2_shape[1],p2_shape[2],p2_shape[3])  #could be wrong here
    #print("dA3 shape", dA3.shape)
    
    # activation_backward(da, a,z,function_index=1,alp=1)
    dA2_prev = pooling_backward_one_layer(dA3, cache["p_cache2"], "max")
    dZ2 = activation_backward(dA2_prev,cache["a2"], cache["z2"], 1)
    dA2, dw2, db2 = convolution_backward_one_layer(dZ2, cache["c_cache2"])

    dA1_prev = pooling_backward_one_layer(dA2, cache["p_cache1"], "max")
    dZ1 = activation_backward(dA1_prev, cache["a1"], cache["z1"], 1)
    dA1, dw1, db1 = convolution_backward_one_layer(dZ1, cache["c_cache1"])

 
    #update the value instead of making a copy here?

    #update weights    
    parameters["w1"] -= learning_rate*dw1
    parameters["w2"] -= learning_rate*dw2
    parameters["w3"] -= learning_rate*dldw_3
    #parameters["w4"] -= learning_rate*dldw_4

    #update bias 
    parameters["b1"] -= learning_rate*db1
    parameters["b2"] -= learning_rate*db2
    parameters["b3"] -= learning_rate*dldb_3
    #parameters["b4"] -= learning_rate*dldb_4

   
    return parameters

In [26]:
def training(parameters, learning_rate = 0.01, epochs = 1, mini_batch_size = 50):
       
    total_size = X_train.shape[0]
    iter_per_epoch = math.ceil(total_size/mini_batch_size)
    for i in range(epochs):
        cost = 0
        for j in range(iter_per_epoch):
            
            begin = j * mini_batch_size            
            if(j < iter_per_epoch -1):
                end = begin + mini_batch_size
            else:
                end = total_size
            #print(begin, end)
            
            pred, All_cache = forward_propagation(X_train[begin:end,...], parameters)
            
            batch_cost = calculate_cross_entropy_loss(pred,begin,end)
            print("iteration",j,"batch_cost", batch_cost)
            cost = cost + batch_cost * (end-begin)
            
            #def backward_propagation(parameters, cache, learning_rate = 0.01):
            parameters = backward_propagation(parameters,All_cache,begin,end,learning_rate)
        
        
        #after one epoch
        cost = cost/total_size
        print("-----> epoch: ", i, ", total cost: ",cost)
        
    print("All done!!! yaaay")
            
    return parameters, pred



# print("prediction \n", pred)
# print("true \n",Y_train[begin: end])
# cost = calculate_cross_entropy_loss(pred, begin,end)
# print("cost",cost)
# predict_test = np.argmax(a3_test, axis=0)
# test_length = len(predict_test)
# test_acc = accuracy(predict_test.reshape(test_length,1),y_test)

In [27]:
# do not run this, be careful!!!!
#parameters = initialization(42) 

In [28]:
#calling the train function
#model_para, predic = training(parameters)


In [29]:
# path_f = "para_Bijie_Qiu.dat"
# save_model(model_para, path_f)

In [38]:
path_f_2 = "para_Bijie_Qiu2.dat"
#paras = load_model(path_f)
paras = load_model(path_f_2)
print_model(paras)
new_paras, predicts = training(paras,epochs = 5,learning_rate = 0.1,mini_batch_size=100) #learning_rate = 0.01
#save_model(new_paras, path_f_2) 

relu max relu max sigmoid sigmoid
b1 (1, 1, 1, 8)
b2 (1, 1, 1, 16)
b3 (1, 6)
b4 (1, 6)
w1 (4, 4, 3, 8)
w2 (4, 4, 8, 16)
w3 (4096, 6)
w4 (256, 6)
iteration 0 batch_cost 2.70841660531
iteration 1 batch_cost 2.72387615011
iteration 2 batch_cost 2.70168580797
iteration 3 batch_cost 2.72427196487
iteration 4 batch_cost 2.71309212488
iteration 5 batch_cost 2.72142140315
iteration 6 batch_cost 2.71546507172
iteration 7 batch_cost 2.72395467484
iteration 8 batch_cost 2.656985792
-----> epoch:  0 , total cost:  2.71535557961
iteration 0 batch_cost 2.70493158253
iteration 1 batch_cost 2.72138971349
iteration 2 batch_cost 2.69804244767
iteration 3 batch_cost 2.72200088315
iteration 4 batch_cost 2.71027627449
iteration 5 batch_cost 2.71925628926
iteration 6 batch_cost 2.71303479087
iteration 7 batch_cost 2.72201242171
iteration 8 batch_cost 2.65143249115
-----> epoch:  1 , total cost:  2.71264382374
iteration 0 batch_cost 2.70225973155
iteration 1 batch_cost 2.71959610357
iteration 2 batch_cost 2.

In [37]:
save_model(new_paras, path_f_2) 

file saved
